In [0]:

%pip install databricks-vectorsearch langchain==0.1.0 mlflow==2.10.0 databricks-sdk --upgrade --quiet

In [0]:
dbutils.library.restartPython()


## Industrial Automation RAG Chatbot
#### Quick Setup with An Existing Vector Search Index

**What this notebook does:**

1. Connects an existing Vector Search index containing Technical documentation on OPC UA
2. Creates a RAG chain with Databricks foundation models
3. Tests the chatbot
4. Registers the model to Unity Catalog
5. Deploys as a REST API endpoint

In [0]:
VECTOR_SEARCH_ENDPOINT = "opcua_manual_siemens_vector_search_endpoint"
VECTOR_SEARCH_INDEX = "workspace.default.opcua_manual_index"
TEXT_COLUMN = "text"

CATALOG_NAME = "workspace"
SCHEMA_NAME = "default"
MODEL_NAME = "opc_ua_rag_agent"
ENDPOINT_NAME = "opc_ua_rag_agent"

# -------------------------------------------------------------
# Foundation model options:
# - databricks-dbrx-instruct ??
# - databricks-meta-llama-3-1-70b-instruct
# - databricks-mixtral-8x7b-instruct
# --------------------------------------------------------------
FOUNDATION_MODEL = "databricks-meta-llama-3-3-70b-instruct"
EMBEDDING_MODEL = "databricks-gte-large-en"

print("Configuration loaded!")
print(f"Vector Search Index: {VECTOR_SEARCH_INDEX}")
print(f"Foundation Model: {FOUNDATION_MODEL}")


from databricks.vector_search.client import VectorSearchClient
from langchain.vectorstores import DatabricksVectorSearch
from langchain.embeddings import DatabricksEmbeddings


vsc = VectorSearchClient(disable_notice=True)


print(f"Connecting to index: {VECTOR_SEARCH_INDEX}...")
vs_index = vsc.get_index(
    endpoint_name=VECTOR_SEARCH_ENDPOINT,
    index_name=VECTOR_SEARCH_INDEX
)


embedding_model = DatabricksEmbeddings(endpoint=EMBEDDING_MODEL)


retriever = DatabricksVectorSearch(
    vs_index,
    text_column=TEXT_COLUMN,
    embedding=embedding_model,
    columns=[TEXT_COLUMN, "page"]
).as_retriever(
    search_kwargs={
        "k": 3
    }
)

print("Successfully connected to Vector Search index!")
print(f"Retriever configured to fetch top 3 most relevant chunks")


from langchain.chat_models import ChatDatabricks


chat_model = ChatDatabricks(
    endpoint=FOUNDATION_MODEL,
    max_tokens=500,
    temperature=0.1
)

print(f"Foundation model '{FOUNDATION_MODEL}' configured!")
print(f"   Max tokens: 500")
print(f"   Temperature: 0.1")


from langchain.prompts import PromptTemplate


PROMPT_TEMPLATE = """You are an expert assistant for industrial automation systems, specializing in protocols and technologies including:
- OPC UA (Open Platform Communications Unified Architecture)
- Modbus TCP (Modbus over TCP/IP)
- MQTT (Message Queuing Telemetry Transport)
- Matter (IoT connectivity standard)
- HART (Highway Addressable Remote Transducer Protocol)

Your role is to provide accurate, technical answers based on the documentation provided.

Use the following context from the documentation to answer the user's question:

{context}

User Question: {question}

Instructions:
- Provide clear, technical answers with specific details
- If you cannot find the answer in the context, say "I don't have information about that in the documentation."
- Do not make up or infer information not present in the context
- When discussing protocols, mention their key characteristics
- Include relevant technical specifications when available

Answer:"""

prompt = PromptTemplate(
    template=PROMPT_TEMPLATE,
    input_variables=["context", "question"]
)

print("Prompt template created...")

from langchain.chains import RetrievalQA

rag_chain = RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

print("RAG chain created successfully...")
print("\nChain components:")
print(f"  Retriever: {VECTOR_SEARCH_INDEX}")
print(f"  LLM: {FOUNDATION_MODEL}")
print(f"  Strategy: Stuff (All context in one prompt)")

In [0]:
import warnings
warnings.filterwarnings('ignore')


test_questions = [
    "What is OPC UA and how does it work?",
    "Explain the main features of Modbus TCP",
    "How does MQTT differ from OPC UA?",
    "What are the security features in these protocols?"
]

print("Testing chatbot with sample questions...\n")

for i, question in enumerate(test_questions, 1):
    print(f"{'-'*80}")
    print(f"Test {i}/{len(test_questions)}")
    print(f"{'-'*80}")
    print(f"Question: {question}\n")
    
    try:
        result = rag_chain.invoke({"query": question})
        
        print(f"Answer:")
        print(f"{result['result']}\n")
        
        # ------------------------------------------------------
        # Print sources
        # ------------------------------------------------------
        if result.get('source_documents'):
            print(f"Sources ({len(result['source_documents'])} documents):")
            for j, doc in enumerate(result['source_documents'], 1):
                source = doc.metadata.get('page', 'Unknown source')

                # ------------------------------------------------------
                # Clean up the source path for better readability
                # ------------------------------------------------------
                if '/' in source:
                    source = source.split('/')[-1]  # Get just the filename
                preview = doc.page_content[:100].replace('\n', ' ') + "..." if len(doc.page_content) > 100 else doc.page_content.replace('\n', ' ')
                print(f"  {j}. {source}")
                print(f"     {preview}")
        else:
            print(f"No source documents returned")
        
        print("\n")
        
    except Exception as e:
        print(f"Error: {str(e)}\n")
        print(f"Skipping to next question...\n\n")
        continue

#### Sample questions related to OPC UA and Industrial Automation

In [0]:
question = "What is OPC UA?" 

print(f"Question: {question}\n")

result = rag_chain({"query": question})

print(f"Answer:\n{result['result']}\n")

print(f"\nRetrieved from {len(result['source_documents'])} source(s):")
for i, doc in enumerate(result['source_documents'], 1):
    print(f"\n{i}. Source: {doc.metadata.get('page', 'Unknown')}")
    print(f"   Content preview: {doc.page_content[:200]}...")

In [0]:
question = "How can I setup security policies correctly?" 

print(f"Question: {question}\n")

result = rag_chain({"query": question})

print(f"Answer:\n{result['result']}\n")

print(f"\nRetrieved from {len(result['source_documents'])} source(s):")
for i, doc in enumerate(result['source_documents'], 1):
    print(f"\n{i}. Source: {doc.metadata.get('page', 'Unknown')}")
    print(f"   Content preview: {doc.page_content[:200]}...")